## Imports

In [5]:
# Add parent directory to path for local imports
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as T
import bcolz
import time
from torch.utils.data import Dataset, DataLoader, sampler
%matplotlib inline

In [6]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

Using gpu: True 


## Data processing

In [ ]:
transform = T.Compose([
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = dset.ImageFolder('../Dataset/train', transform=transform)
val_dataset = dset.ImageFolder('../Dataset/val', transform=transform)
test_dataset = dset.ImageFolder('../Dataset/test', transform=transform)

In [ ]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)
print("Number of training examples {}, validation examples {}, testing examples {}".format(train_size, val_size, test_size))

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=6)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=6)

## Ensemble Model

In [ ]:
class FoodEnsemble(nn.Module):
    def __init__(self, resnet, densenet):
        self.resnet = resnet
        self.densenet = densenet
        self.alpha = 0.5
    def forward(self, x):
        with torch.no_grad():
            a1 = self.resnet(x)
            a2 = self.densenet(x)
        return self.alpha * a1 + (1 - self.alpha) * a2
        

resnet = gpu(torch.load("../saved_models/food/res18/res18.dat"))
densenet = gpu(torch.load("../saved_models/food/densenet/densenet.dat"))
FoodEnsemble(resnet, densenet)

In [7]:
def check_accuracy(model):
    num_correct, num_samples, total_loss = 0, 0, 0
    model.eval()
    batches = val_dataloader
    with torch.no_grad():
        for x, y in batches:
            x, y = gpu(x), gpu(y)                
            scores = model(x)
            loss = F.cross_entropy(scores, y) 
            _, preds = torch.max(scores.data, 1)
            total_loss += loss.data.item()
            num_correct += torch.sum(preds == y.data)
            num_samples += preds.size(0)
        average_loss = total_loss / num_samples
        acc = num_correct / num_samples
    print('Validation Loss: {:.4f} Got {} / {} correct {:.2f}%'.format(
        average_loss, num_correct, num_samples, 100 * acc))


In [ ]:
%%time
check_accuracy(model)

Epoch: 1 Training Loss: 0.0414 Got 27637 / 66071 correct. Acc: 41.83%
Epoch: 1 Validation Loss: 0.0295 Got 5813 / 11016 correct 52.77%
Epoch: 2 Training Loss: 0.0268 Got 37798 / 66071 correct. Acc: 57.21%
Epoch: 2 Validation Loss: 0.0262 Got 6261 / 11016 correct 56.84%
Epoch: 3 Training Loss: 0.0239 Got 40254 / 66071 correct. Acc: 60.93%
Epoch: 3 Validation Loss: 0.0250 Got 6499 / 11016 correct 59.00%
Epoch: 4 Training Loss: 0.0224 Got 41382 / 66071 correct. Acc: 62.63%
Epoch: 4 Validation Loss: 0.0248 Got 6508 / 11016 correct 59.08%
Epoch: 5 Training Loss: 0.0214 Got 42447 / 66071 correct. Acc: 64.24%
Epoch: 5 Validation Loss: 0.0248 Got 6533 / 11016 correct 59.30%
Epoch: 6 Training Loss: 0.0206 Got 43204 / 66071 correct. Acc: 65.39%
Epoch: 6 Validation Loss: 0.0245 Got 6590 / 11016 correct 59.82%


In [ ]:
# Save the model.
PATH = "../saved_models/food/res18/res18.dat"
torch.save(model, PATH)
torch.save({'loss': LOSS, 'acc': ACC, 'loss_v': LOSS_V, 'acc_v': ACC_V}, '../saved_models/food/res18/res18-history.pt')

## Evaluation

In [ ]:
# Load the model.
PATH = "../saved_models/food/res18/res18.dat"
loaded_model = torch.load(PATH)
loaded_model = gpu(loaded_model)
model = loaded_model
history = torch.load('../saved_models/food/res18/res18-history.pt')
LOSS, ACC, LOSS_V, ACC_V = (history['loss'], history['acc'], history['loss_v'], history['acc_v'])

In [ ]:
# Loss and Accuracy Curves
from utils.plots import plot_loss_and_accuracy_curves
plot_loss_and_accuracy_curves('ResNet18', LOSS, ACC, LOSS_V, ACC_V)

In [ ]:
# Evaluation: Top-1 Accuracy

def compute_top_1_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for data in test_dataloader:
            images, labels = data
            images = gpu(images)
            labels = gpu(labels)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Top 1 Accuracy of the network on the all test images: %.2f %%' % (
        100 * correct / total))

compute_top_1_accuracy(model)

In [ ]:
# Evaluation: Top-5 Accuracy

def compute_top_5_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for data in test_dataloader:
            images, labels = data
            images = gpu(images)
            labels = gpu(labels)
            outputs = model(images)
            _, predicted = torch.topk(outputs.data, 5, dim=1)
            total += labels.size(0)
            for i in range(predicted.shape[0]):
                top_5_predictions = predicted[i]
                label = labels[i]
                if label in top_5_predictions:
                    correct += 1
    print('Top 5 Accuracy of the network on the all test images: %.2f %%' % (
        100 * correct / total))
    
compute_top_5_accuracy(model)

In [ ]:
import sklearn.metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
# Store y_pred and y_test on the test set for evaluation.

y_pred = []
y_test = []
with torch.no_grad():
    model.eval()
    for data in test_dataloader:
        images, labels = data
        images = gpu(images)
        labels = gpu(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy().tolist())
        y_test.extend(labels.cpu().numpy().tolist())

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(cnf_matrix)
print(cnf_matrix[0][0], sum(cnf_matrix[0]))

In [ ]:
# Visualizing the confusion matrix 

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = cnf_matrix.tolist()
df_cm = pd.DataFrame(array, index = [i for i in range(172)],
                  columns = [i for i in range(172)])
plt.figure(figsize = (100,100))
sn.heatmap(df_cm, annot=True)

In [ ]:
# Evaluation: Compute several statistics such as accuracy, precision, F1-score, and produce a classification report

FOOD_LIST_PATH = '../VireoFood172/SplitAndIngreLabel/FoodList.txt'
food_names = [] # List of length 172 where index is the food label, and value is the food name.
with open(FOOD_LIST_PATH) as fp:
    food_names = fp.read().splitlines()
    
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=food_names))

In [ ]:
print(len(y_test))

In [ ]:
# Class 0 Statistics

TP = 0
FP = 0
FN = 0
for pred,label in zip(y_pred, y_test):
    if pred == 0 and label == 0:
        TP += 1
    if pred == 0 and label != 0:
        FP += 1
    if pred != 0 and label == 0:
        FN += 1
print("Class 0 ({}) Statistics".format(food_names[0]))
print("True Positives: {}".format(TP))
print("False Positives: {}".format(FP))
print("False Negatives: {}".format(FN))
print("Precision: {}".format(TP / (TP + FP)))
print("Recall: {}".format(TP / (TP + FN)))